# OSM Kent Railways Import

Go to (Overpass Turbo)[https://overpass-turbo.eu/] and export railway route data via the wizard with route=train


In [2]:
import configparser
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import webbrowser
from sqlalchemy import create_engine


DATA_DIR = os.path.join('D:\\Data\\TPAM\\')
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [2]:

osmhistory_railroutes = gpd.read_file(DATA_DIR + "osmhistory_railroutes.geojson")
closed_routes = gpd.read_file(DATA_DIR + "Closed UK Railways.kml", driver='KML')

c:\Users\paulf\anaconda3\envs\GeoDjango\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [3]:
def create_folium_map(routes_to_map):
    m = folium.Map(location=[52.48, 0], zoom_start= 7, tiles='cartodbpositron')
    folium.GeoJson(data=routes_to_map["geometry"]).add_to(m)
    m.save('temporary_map.html')
    webbrowser.open('temporary_map.html', new=2)
    return

In [4]:
def print_df_info(df):
    #routes.to_csv('df.csv', index=False)
    print(type(df))
    print(df.head())
    print(df.crs)
    print(df.total_bounds)
    print(df.info())
    df.plot(figsize=(6, 6))
    plt.show()
    return

In [4]:

config = configparser.ConfigParser()
KEYS_DIR = os.path.join("D:\\Data", "API_Keys")
config.read(os.path.join(KEYS_DIR, "TPAMWeb.ini"))
db_pswd = config['MySQL']['p']
engine = create_engine('postgresql://postgres:' + db_pswd + '@localhost/TPAM')


In [4]:
routes = gpd.read_file(DATA_DIR + "osm_railroutes.geojson")
routes.to_postgis(name="locations_routes_geo_osm", if_exists='append', index=False, con=engine)
create_folium_map(routes)
print_df_info(routes)

NameError: name 'create_folium_map' is not defined

In [5]:
routes = gpd.read_file(DATA_DIR + "osmhistory_railroutes.geojson")
routes.to_postgis(name="locations_routes_geo_osmhistory", if_exists='append', index=False, con=engine)
create_folium_map(routes)
print_df_info(routes)

NameError: name 'create_folium_map' is not defined

### Closed UK Railways
Data download from a map [created by a contributor on Google maps](https://www.google.com/maps/d/viewer?ie=UTF8&oe=UTF8&msa=0&mid=1-qtKdwA-_7Y1msH5nlUbVKMjdMk&ll=51.17557904660408%2C-0.34163366478264123&z=9)
Note that in Postgres/Django the column "Name" has been changed to "name". Needs to be changed back temporarily for this load to work.

In [10]:
closed_routes = gpd.read_file(DATA_DIR + "Closed UK Railways.kml", driver='KML')
closed_routes.head()
closed_routes.to_postgis(name="locations_routes_geo_closed", if_exists='append', index=False, con=engine)
# create_folium_map(closed_routes)
# print_df_info(closed_routes)

In [8]:
admin_boundaries = gpd.read_file("D:\\Data\\UK_Admin_Boundaries\\" + "Counties_and_Unitary_Authorities__December_2019__Boundaries_UK_BFE.shp")
admin_boundaries.to_postgis(name="locations_UK_admin_boundaries", if_exists='append', index=False, con=engine)
# create_folium_map(admin_boundaries)
# print_df_info(admin_boundaries)